In [7]:
%load_ext tensorboard

First, we'd need to upload dataset to GDrive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

data_path = "TODO"

In [8]:
import jax
from absl_extra import logging_utils
import temporal_fusion_transformer as tft
from temporal_fusion_transformer.config import get_config

logging_utils.setup_logging(log_level="INFO")
jax.devices()

[CpuDevice(id=0)]

In [9]:
config = get_config("electricity")
config

fixed_params:
  input_known_categorical_idx:
  - 3
  - 4
  - 5
  - 6
  input_known_real_idx:
  - 1
  - 2
  input_observed_idx: []
  input_static_idx:
  - 0
  known_categories_sizes:
  - 31
  - 24
  - 7
  - 12
  num_encoder_steps: 168
  num_outputs: 1
  static_categories_sizes:
  - 369
  total_time_steps: 192
hyperparams:
  dropout_rate: 0.1
  latent_dim: 160
  num_attention_heads: 4
  num_decoder_blocks: 1
  quantiles:
  - 0.1
  - 0.5
  - 0.9
optimizer:
  clipnorm: 0.1
  decay_alpha: 0.05
  decay_steps: 0.8
  ema: 0.99
  learning_rate: 0.001
prng_seed: 69
shuffle_buffer_size: 1024

In [ ]:
tft.scripts.train_on_single_device(
    data_dir=data_path,
    batch_size=64,
    experiment_name="electricity",
    mixed_precision=True,
    config=config,
    save_path="TODO",
)